In [2]:
import numpy as np

import sqlalchemy
from sqlalchemy.ext.automap import automap_base
from sqlalchemy.orm import Session
from sqlalchemy import create_engine, func

from flask import Flask, jsonify


#################################################
# Database Setup
#################################################
engine = create_engine("sqlite:///Resources/hawaii.sqlite")

# reflect an existing database into a new model
Base = automap_base()
# reflect the tables
Base.prepare(engine, reflect=True)
station = Base.classes.station
measurement = Base.classes.measurement

#################################################
# Flask Setup
#################################################
app = Flask(__name__)


#################################################
# Flask Routes
#################################################

@app.route("/")
def welcome():
    """List all available api routes."""
    return (
        f"Available Routes:<br/>"
        f"/api/v1.0/precipitation<br/>"
        f"/api/v1.0/stations<br/>"
        f"/api/v1.0/tobs<br/>"
        f"/api/v1.0/<start><br/>"
        f"/api/v1.0/<start><end><br/>"
    )


@app.route("/api/v1.0/precipitation")
def precipitation():
    # Create our session (link) from Python to the DB
    session = Session(engine)


    results = session.query(measurement.date,measurement.prcp).filter(measurement.date.between('2016-08-23', '2017-08-23'))

    session.close()

    all_precipitation = []
    for row in results:
        precipitation_dict = {}
        precipitation_dict[row[0]] = row[1]
        all_precipitation.append(precipitation_dict)

    return jsonify(all_precipitation)

@app.route("/api/v1.0/stations")
def names():
    session = Session(engine)
    results = session.query(station.station).all()
    session.close()
    all_names = list(np.ravel(results))
    return jsonify(all_names)

@app.route("/api/v1.0/tobs")
def obs():
    session = Session(engine)
    tab1=session.query(measurement.date,measurement.tobs).filter(measurement.date.between('2016-08-23', '2017-08-23'))
    session.close()
    all_tobs = list(tab1)
    return jsonify(all_tobs)


@app.route("/api/v1.0/<start>")
def min_max_avg_start(start):
    session = Session(engine)
    tab1=session.query(func.min(measurement.tobs),func.max(measurement.tobs),func.avg(measurement.tobs)).\
    filter(measurement.date >= start).first()
    session.close()
    all_tobs = list(tab1)
    return jsonify(all_tobs)

@app.route("/api/v1.0/<start><end>")
def min_max_avg_start_end(start,end):
    session = Session(engine)
    tab1=session.query(func.min(measurement.tobs),func.max(measurement.tobs),func.avg(measurement.tobs)).\
    filter(measurement.date.between(start,end)).first()
    session.close()
    all_tobs = list(tab1)
    return jsonify(all_tobs)


if __name__ == '__main__':
    app.run(debug=False)


 * Serving Flask app "__main__" (lazy loading)
 * Environment: production
   Use a production WSGI server instead.
 * Debug mode: off


 * Running on http://127.0.0.1:5000/ (Press CTRL+C to quit)
127.0.0.1 - - [21/Feb/2020 19:06:51] "GET / HTTP/1.1" 200 -
127.0.0.1 - - [21/Feb/2020 19:07:21] "GET /api/v1.0/2016-08-232017-08-23 HTTP/1.1" 200 -
127.0.0.1 - - [21/Feb/2020 19:07:38] "GET /api/v1.0/2016-08-23 HTTP/1.1" 200 -
